In [ ]:
'''
Known 
1. ICM fitting result
RGH80_reg0_spectral_results.txt 
NH     0.0134
REDSHIFT      0.0361
tbabs * apec

2. BKG fitting result
data: bkgregion + rosat
cxbpars.txt
lhb    8.78236e-07   5.26693e-08
ght    0.168069   0.00567776
ghn    1.80237e-06   1.135e-07
cxb    1.12266e-06   3.34785e-08
NH     0.0134
REDSHIFT      0.0361
apec + tbabs*(apec+apec+pow)

3. My result

1) lower lhb norm
2) higher gh temperature, slightly higher gh norm
3) lower cxb norm

   3    3   apec       kT         keV      0.127901     +/-  0.0          
   6    3   apec       norm                8.07050E-07  +/-  0.0          
  12    6   apec       kT         keV      0.199997     +/-  0.0          
  15    6   apec       norm                1.83883E-06  +/-  0.0          
  17    7   powerlaw   norm                9.61054E-07  +/-  0.0  

Total fit statistic                          2844.32     with 2391 d.o.f.

'''



1 set sky bkg model

In [1]:
# > bkg_all.xcm
abun aspl
statistic cstat

# Cstat should be used here. Since rass spec is counts spectrum now, 
# therefore it obeys poisson statistics. 
# Then cstat is better for spectrum fitting. 

data 1:1 mos1S001-obj-RGH80_bkg-grp.pi
data 2:2 mos2S002-obj-RGH80_bkg-grp.pi
data 3:3 pnS003-obj-RGH80_bkg-grp.pi
res 1 mos1S001-RGH80_bkg.rmf
res 2 mos2S002-RGH80_bkg.rmf
res 3 pnS003-RGH80_bkg.rmf
arf 1 mos1S001-RGH80_bkg.arf
arf 2 mos2S002-RGH80_bkg.arf
arf 3 pnS003-RGH80_bkg.arf
backgrnd 1 mos1S001-back-RGH80_bkg.pi
backgrnd 2 mos2S002-back-RGH80_bkg.pi
backgrnd 3 pnS003-back-RGH80_bkg.pi

ign **-**:**-0.3,7.-**
data 4:4 ../rosat.pi
res 4 ../rosat.rsp
ign bad
cpd /xs
pl ldat

SyntaxError: invalid syntax (1054460081.py, line 2)

In [ ]:
'''
BACKSCAL 
mos1: 2.136593E+08 pix # 148.38 arcmin2
mos2: 2.114375E+08 pix # 146.83 arcmin2
pn: 1.689255E+08 pix # 117.31 arcmin2
'''

In [6]:
# get backscal in the pi files
from astropy.io import fits
import os

homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/reduction/SDSSTG3460/SDSSTG3460_bkg'
os.chdir(homepath)

# every pixel length is 0.05 arcsec

for name in ['mos1S001', 'mos2S002', 'pnS003-0', 'pnS003-4']:
    f = fits.open(f'{name}-bkg.pi')
    print(f'''{name}: {f[1].header['BACKSCAL'] * (0.05/60) ** 2}''')

mos1S001: 93.5147277777778
mos2S002: 127.54238888888891
pnS003-0: 121.32156736111114
pnS003-4: 121.32156736111114


In [ ]:
# set bkg model
# >bkg_mdl.xcm 
mo const*const*(apec + tbabs*(apec+apec+pow))

In [ ]:


# extract region area
new 1 148.38
new 18 146.83
new 35 117.31
new 52 1 
# rosat spectrum has been normalized to 1 arcmin2 ? yes!
free 1,18,35,52

# inst variance
new 2 1
new 19 1
new 36 1
new 53 1
free 2,19,36,53

# LHB
new 3 0.11
new 6 9e-7
free 3 # keep fixed

# nH
new 7 0.0134
free 7

# GH_soft
# not always need to consider
new 8 0.05
new 11 0
free 8,11
# maybe thaw later

# GH_hard 
# norm can be spatially variable
new 12 0.2
new 15 2e-6
free 12 
# thaw later

# cxb
new 16 1.46
new 17 1.1e-6
free 16 
# always needs to be frozen 



2 set alsi model

In [ ]:
res 2:1 mos1S001-RGH80_bkg.rmf
res 2:2 mos2S002-RGH80_bkg.rmf
res 2:3 pnS003-RGH80_bkg.rmf
arf 2:1 mos1S001-RGH80_bkg.arf
arf 2:2 mos2S002-RGH80_bkg.arf
arf 2:3 pnS003-RGH80_bkg.arf
backgrnd 2:1 mos1S001-back-RGH80_bkg.pi
backgrnd 2:2 mos2S002-back-RGH80_bkg.pi
backgrnd 2:3 pnS003-back-RGH80_bkg.pi

mo 2:alsi const*const*(gauss + gauss + gauss + gauss)


In [2]:
# set alsi parameter
print(
'''
new alsi:1 148.38
new alsi:15 146.83
new alsi:29 117.31
free alsi:1,15,29

new alsi:2 1
new alsi:16 1
new alsi:30 1
free alsi:2,16,30

# Mg Ka 1.253
new alsi:3 1.253
new alsi:4 0
new alsi:5 1e-6
free alsi:3,4

new alsi:17=alsi:3
new alsi:18=alsi:4
new alsi:19=alsi:5

new alsi:31=alsi:3
new alsi:32=alsi:4
new alsi:33=alsi:5


''')




new alsi:1 148.38
new alsi:15 146.83
new alsi:29 117.31
free alsi:1,15,29

new alsi:2 1
new alsi:16 1
new alsi:30 1
free alsi:2,16,30

# Mg Ka 1.253
new alsi:3 1.253
new alsi:4 0
new alsi:5 1e-3
free alsi:3,4

new alsi:17=alsi:3
new alsi:18=alsi:4
new alsi:19=alsi:5

new alsi:31=alsi:3
new alsi:32=alsi:4
new alsi:33=alsi:5



In [4]:
# Al Ka 1.486
step = 3
print(
f'''
new alsi:{step+3} 1.486
new alsi:{step+4} 0
new alsi:{step+5} 1e-5
free alsi:{step+3}, {step+4}

new alsi:{step+17}=alsi:{step+3}
new alsi:{step+18}=alsi:{step+4}
new alsi:{step+19}=alsi:{step+5}

new alsi:{step+31}=alsi:{step+3}
new alsi:{step+32}=alsi:{step+4}
new alsi:{step+33}=alsi:{step+5}
'''
)



new alsi:6 1.486
new alsi:7 0
new alsi:8 1e-5
free alsi:6, 7

new alsi:20=alsi:6
new alsi:21=alsi:7
new alsi:22=alsi:8

new alsi:34=alsi:6
new alsi:35=alsi:7
new alsi:36=alsi:8



In [9]:
# Al Kb 1.557, mos only
step = 6
print(
f'''
new alsi:{step+3} 1.557
new alsi:{step+4} 0
new alsi:{step+5} 1e-5
free alsi:{step+3}, {step+4}

new alsi:{step+17}=alsi:{step+3}
new alsi:{step+18}=alsi:{step+4}
new alsi:{step+19}=alsi:{step+5}

'''
)



new alsi:9 1.557
new alsi:10 0
new alsi:11 1e-5
free alsi:9, 10

new alsi:23=alsi:9
new alsi:24=alsi:10
new alsi:25=alsi:11




In [10]:
# Cr Kb 5.947, pn only
step = 6
print(
f'''
new alsi:{step+31} 5.947
new alsi:{step+32} 0
new alsi:{step+33} 1e-5
free alsi:{step+31},{step+32}

'''
)



new alsi:37 5.947
new alsi:38 0
new alsi:39 1e-5
free alsi:37,38




In [11]:
# set last gauss of pn as 0
step = 9
print(
f'''
new alsi:{step+31} 0
new alsi:{step+32} 0
new alsi:{step+33} 0
free alsi:{step+31},{step+32},{step+33}

'''
)


new alsi:40 0
new alsi:41 0
new alsi:42 0
free alsi:40,41,42




In [ ]:
# thaw the line center
thaw alsi:3,6,9

4 set pn oot model

In [ ]:
res 4:3 pnS003-RGH80_bkg.rmf
arf 4:3 pnS003-RGH80_bkg.arf
backgrnd 4:1 mos1S001-back-RGH80_bkg.pi
backgrnd 4:2 mos2S002-back-RGH80_bkg.pi
backgrnd 4:3 pnS003-back-RGH80_bkg.pi


mo 4:oot const*const*(apec + tbabs*(apec+apec+pow) + gauss + gauss + gauss + gauss + bknpow)

mo 4:oot const*const*()


3 set spf model

In [ ]:
resp 3:1 mos1-diag.rsp 
resp 3:2 mos2-diag.rsp 
resp 3:3 pn-diag.rsp 
model 3:spf const*const*(bknpow)

In [ ]:
# set area const
new spf:1 148.38
new spf:7 146.83
new spf:13 117.31
free spf:1,7,13

new spf:2 1
new spf:8 1
new spf:14 1
free spf:2,8,14

# set bknpow index
new spf:3 0.7
new spf:9 0.7
new spf:15 0.7

new spf:5 0.7
new spf:11 0.7
new spf:17 0.7
free spf:3,9,15,5,11,17

# set breakE
free spf:4,10,16

# estimate spf norm
new spf:6 1e-5
new spf:12 1e-5
new spf:18 1e-5
